Lesson 12
# LLMを活用したExcel操作の自動化

## openpyxl

In [2]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [4]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()


,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [5]:
# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [6]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())


この売上データを分析するために、以下のいくつかの主要なポイントを考察します。

### 1. 売上の全体トレンド
- **日付別の売上分析**: 売上日ごとの売上合計（売上額 × 数量）を集計し、全体のトレンドを把握することが重要です。月ごとの売上推移や特定の日の値動きなどが見られます。
- **カテゴリー別の売上分析**: 食品、衣服、日用品などのカテゴリー別に売上を集計し、どのカテゴリーが最も売上を上げているかを確認します。

### 2. 商品別の売上分析
- **人気商品の特定**: 単価、数量、売上の値を元に、最も売上が高い商品や、数量が多く売れた商品を特定します。この分析により、顧客に人気のある商品や利益率の高い商品を把握することができます。
- **原価に対する利益の可視化**: 商品ごとの利益（売上 - 原価）を計算し、利益率の高い商品を特定します。これにより、利益を最大化するための戦略を立てることができます。

### 3. 時間帯やシーズン別トレンド
- **季節変動**: 特定の季節（例えば、夏にTシャツが売れる、冬に衣服が売れるなど）の傾向を分析します。販促のタイミングや新商品の投入時期を決定するのに役立ちます。

### 4. マーケティング施策の効果
- 過去に行ったプロモーションやキャンペーンの結果を分析し、顧客の反応や売上への影響を評価します。売上が大幅に伸びた場合、その要因を特定することで今後の施策に活かせます。

### 5. 顧客セグメントの分析
- **リピート率や顧客の購入パターン**: 顧客データがあれば、リピート率や一回の購入での平均支出を計算し、顧客ロイヤリティを測定できます。

### まとめ
これらの分析を行うことで、売上データの傾向を把握し、戦略的な意思決定に役立つインサイトを得ることが可能になります。例えば、特定の商品の販売促進を強化したり、利益率の低い商品を見直したり、適切な時期に新商品を投入するなどの施策を設計できます。それにより企業の成長を支援するための具体的なアクションが可能になります。もし具体的なデータや可視化の必要があれば、さらに詳細な分析が可能です。


In [8]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)


                                                   結果
0              この売上データを分析するために、以下のいくつかの主要なポイントを考察します。
1                                                    
2                                    ### 1. 売上の全体トレンド
3   - **日付別の売上分析**: 売上日ごとの売上合計（売上額 × 数量）を集計し、全体のトレ...
4   - **カテゴリー別の売上分析**: 食品、衣服、日用品などのカテゴリー別に売上を集計し、ど...
5                                                    
6                                     ### 2. 商品別の売上分析
7   - **人気商品の特定**: 単価、数量、売上の値を元に、最も売上が高い商品や、数量が多く売...
8   - **原価に対する利益の可視化**: 商品ごとの利益（売上 - 原価）を計算し、利益率の高...
9                                                    
10                               ### 3. 時間帯やシーズン別トレンド
11  - **季節変動**: 特定の季節（例えば、夏にTシャツが売れる、冬に衣服が売れるなど）の傾...
12                                                   
13                                ### 4. マーケティング施策の効果
14  - 過去に行ったプロモーションやキャンペーンの結果を分析し、顧客の反応や売上への影響を評価し...
15                                                   
16                                  ### 5. 顧客セグメントの分析
17  - **リピート率や顧客の購入パターン**: 顧

In [9]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

In [10]:
# ワークフロー化
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')

# 2. データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"

# 3. OpenAI APIの呼び出し
# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"
# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")

処理を開始します。
Excelファイルに分析結果を保存しました。
